# Cleaning Flood Water Extent for Kampala and Addis Ababa

This notebook cleans the file of monthly flood water extent by;
- Removing unnecessary columns.
- Adding in values for the missing years
- Sorting the rows so each row has coordinates for each city of the same year.
- Re-arranging the variables in the order

In [ ]:
import pandas as pd

df = pd.read_csv("Monthly_Flood_Water_Extent_Kampala_Addis_2015_2025_km2.csv")

In [2]:
df.head()

,system:index,city,flood_water_extent (km^2),month,source,.geo
0,0_0,Kampala,195.380092,2015-01,Sentinel-1,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,0_1,Kampala,207.921894,2015-02,Sentinel-1,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,0_2,Kampala,140.035708,2015-03,Sentinel-1,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,0_3,Kampala,112.024489,2015-04,Sentinel-1,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,0_4,Kampala,114.628558,2015-05,Sentinel-1,"{""type"":""MultiPoint"",""coordinates"":[]}"


In [3]:
df.columns

Index(['system:index', 'city', 'flood_water_extent (km^2)', 'month', 'source',
       '.geo'],
      dtype='object')

In [ ]:
# Extracting the year from the 'month' column
df["year"] = df["month"].str.extract(r"(\d{4})").astype(int)

# Removing unnecessary columns
df = df.drop(columns=["system:index", "month", "source", ".geo"])

# Grouping by year and city to sum the flood water extent
yearly_flood_water_extent = (
    df.groupby(["year", "city"])["flood_water_extent (km^2)"].sum().reset_index()
)

# Sorting the variables so that the year is first and
# city is second then flood water extent
yearly_flood_water_extent = yearly_flood_water_extent.sort_values(
    by=["year", "city"]
).reset_index(drop=True)

In [5]:
yearly_flood_water_extent.head()

,year,city,flood_water_extent (km^2)
0,2015,Addis Ababa,1161.517373
1,2015,Kampala,1789.580176
2,2016,Addis Ababa,1532.617001
3,2016,Kampala,2217.205481
4,2017,Addis Ababa,1377.904746


In [ ]:
# Create a full year range and all cities
years = pd.Series(range(2005, 2026), name="year")
cities = yearly_flood_water_extent["city"].unique()
full_index = pd.MultiIndex.from_product([years, cities], names=["year", "city"])

# Reindex and fill missing combinations with 0
df_filled = (
    yearly_flood_water_extent.set_index(["year", "city"])
    .reindex(full_index, fill_value=0)
    .reset_index()
)

df_filled.head()

,year,city,flood_water_extent (km^2)
0,2005,Addis Ababa,0.0
1,2005,Kampala,0.0
2,2006,Addis Ababa,0.0
3,2006,Kampala,0.0
4,2007,Addis Ababa,0.0


In [7]:
df_filled

,year,city,flood_water_extent (km^2)
0,2005,Addis Ababa,0.000000
1,2005,Kampala,0.000000
2,2006,Addis Ababa,0.000000
3,2006,Kampala,0.000000
4,2007,Addis Ababa,0.000000
5,2007,Kampala,0.000000
6,2008,Addis Ababa,0.000000
7,2008,Kampala,0.000000
8,2009,Addis Ababa,0.000000
9,2009,Kampala,0.000000


In [8]:
# Saving the cleaned data to a new CSV file
df_filled.to_csv(
    "Cleaned_Flood_Water_Extent_Kampala_Addis_2005_2025_km2.csv", index=False
)